In [77]:
import re

from itertools import product

from torch import argmax
from transformers import BertTokenizerFast, BertForTokenClassification

In [83]:
model = BertForTokenClassification.from_pretrained("./BERT_model/checkpoint-2000")
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
tokenizer.add_tokens("(.*?)")

sentences = [
    'Non-zero return code from generate (65); Logfile error in log.generate: ""Pythia8             FATAL /build/atnight/localbuilds/nightlies/21.6/athena/Generators/GeneratorModules/src/GenModule.cxx:56 (StatusCode GenModule::execute()): code 0: this (truncated)',
    'Non-zero return code from AODtoDAOD (65); Logfile error in log.AODtoDAOD: ""StreamDAOD_STDM4                                    FATAL streamObjects failed.""',
    'Found 0 output files for DAOD, expected 8 (found: [])',
    'Fatal error in athena logfile: ""G4 exception at line 56998 (see jobReport for further details)""',
    'AODtoDAOD got a SIGABRT signal (exit code 134); Logfile error in log.AODtoDAOD: ""Core dump from CoreDumpSvc: Event counter: 17596; Run: 284500; Evt: 9315118; Current algorithm: TEST13Kernel; Current Function: unknown""',
    'Data type graph error'
]

tokenized = [tokenizer(sentence, return_tensors='pt') for sentence in sentences]

TODO: Some way to get the labels automatically, if possible

In [84]:
all_labels = ['[PAD]', 'O'] + [f"{prefix}-{label}" for prefix, label in product('BILU', ["Meaningful", "Meaningless"])]
id_to_label = {i: label for i, label in enumerate(all_labels)}

That's a bit of a weird way to get relevant sentence parts.
I had to do it this way because tokenization is, as far as I can tell, irreversible.
So I had to use initial sentence to get the correct spacing.

**That's a hack, will need to do better later**

In [87]:
spaces = re.compile(r'\s+')

for sentence, initial_sentence in zip(tokenized, sentences):
    i = 0
    beginnings, ends = [], []

    output = model(**sentence)
    predictions = argmax(output.logits, dim=2)

    for token, prediction in zip(sentence.tokens(), predictions[0].numpy()):
        if token in ['[CLS]', '[SEP]']:
            continue
        if token.startswith("##"):
            token = token[2:]

        i += initial_sentence[i:].find(token)
        predicted_label = id_to_label[prediction]
        if predicted_label[2:] == "Meaningful":
            beginnings.append(i)
            ends.append(i + len(token))

    print(initial_sentence)
    print(spaces.sub(' ', initial_sentence[beginnings[0]:ends[-1]]), end='\n\n')


Non-zero return code from generate (65); Logfile error in log.generate: ""Pythia8             FATAL /build/atnight/localbuilds/nightlies/21.6/athena/Generators/GeneratorModules/src/GenModule.cxx:56 (StatusCode GenModule::execute()): code 0: this (truncated)
Pythia8 FATAL /build/atnight/localbuilds/nightlies/21.6/athena/Generators/GeneratorModules/src/GenModule.cxx:56 (StatusCode GenModule::execute()): code 0: this (truncated)

Non-zero return code from AODtoDAOD (65); Logfile error in log.AODtoDAOD: ""StreamDAOD_STDM4                                    FATAL streamObjects failed.""
StreamDAOD_STDM4 FATAL streamObjects failed

Found 0 output files for DAOD, expected 8 (found: [])
0 output files for DAOD, expected 8 (found: [

Fatal error in athena logfile: ""G4 exception at line 56998 (see jobReport for further details)""
G4 exception at line 56998 (see jobReport for further details)

AODtoDAOD got a SIGABRT signal (exit code 134); Logfile error in log.AODtoDAOD: ""Core dump from CoreDu